**PROJETO:** SISTEMA IA PARA RECEITAS

**DISCENTES:** André Arcuri Martins, Gabriel Da Silva Souza, Giulia Mota Apinagés Dos Santos, José Vitor Santos Alves, Guilherme De Luca Testoni Neiva Pereira

**1. IMPORTAÇÃO DE BIBLIOTECAS E CARREGAMENTO DO DATASET**

In [ ]:
!pip install isodate
import isodate
import re
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter
from fractions import Fraction
from IPython.display import display

file_path = "recipes.csv"

# Carrega o dataset do Kaggle via KaggleHub
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "irkaal/foodcom-recipes-and-reviews",
  file_path,
)
# Visualiza o DataFrame carregado
df

/tmp/ipython-input-1609321390.py:13: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'foodcom-recipes-and-reviews' dataset.


**2. FUNÇÕES DE PRÉ-PROCESSAMENTO DE INGREDIENTES**
  
* **def limpar_ingrediente:** Função única para limpar e padronizar nomes de ingredientes.
*  **def robust_parse:** Converte uma string bruta de ingredientes/quantidades em uma lista padronizada.
*   **def create_ingredient_list:** remove números, unidades de medida e espaços extras, retornando apenas os nomes dos ingredientes normalizados em letras minúsculas.

In [ ]:
def limpar_ingrediente(ingr_name):
    ingr_name_clean = ingr_name.strip().lower()

    ingr_name_clean = re.sub(r'\d+|\bml\b|\bg\b|\bkg\b|\bl\b|\btbsp\b|\btsp\b', '', ingr_name_clean).strip()

    ingr_name_clean = re.sub(r'\s+', ' ', ingr_name_clean)

    return ingr_name_clean

def robust_parse(s):
    if not isinstance(s, str):
        return []
    match = re.search(r'^c\((.*)\)$', s.strip())
    if not match:
        if s == 'character(0)': return []
        if s.startswith('"') and s.endswith('"'):
            return [s[1:-1]]
        return []
    content = match.group(1)
    items = content.split(',')
    cleaned_items = []
    for item in items:
        item = item.strip()
        if item.startswith('"') and item.endswith('"'):
            item = item[1:-1]
        cleaned_items.append(item)
    return cleaned_items

def create_ingredient_list(row):
    ingredients = robust_parse(row['RecipeIngredientParts'])
    if not ingredients:
        return None

    recipe_list = []
    for ingr_name in ingredients:

        ingr_name_clean = limpar_ingrediente(ingr_name)

        if ingr_name_clean:
            recipe_list.append(ingr_name_clean)

    return recipe_list

df_sample = df.copy()

df_sample['ingredient_list'] = df_sample.apply(create_ingredient_list, axis=1)

df_sample = df_sample.dropna(subset=['ingredient_list'])
df_sample = df_sample[df_sample['ingredient_list'].map(lambda l: l != [])]

print(f"Total de receitas válidas após o parse: {len(df_sample)}")

display(df_sample[['Name', 'ingredient_list']].head())

Criando a lista de ingredientes padronizados...
Total de receitas válidas após o parse: 520208


,Name,ingredient_list
0,Low-Fat Berry Blue Frozen Dessert,"[blueberries, granulated sugar, vanilla yogurt..."
1,Biryani,"[saffron, milk, hot green chili peppers, onion..."
2,Best Lemonade,"[sugar, ""lemons, rind of"", ""lemon, zest of"", f..."
3,Carina's Tofu-Vegetable Kebabs,"[extra firm tofu, eggplant, zucchini, mushroom..."
4,Cabbage Soup,"[plain tomato juice, cabbage, onion, carrots, ..."


**3. LIMPEZA E AJUSTES DE ATRIBUTOS**

**Remoção de colunas irrelevantes ao modelo**

In [ ]:
df_sem_atributos = df_sample.drop(columns=['RecipeId', 'Name', 'AuthorId', 'AuthorName',
                                    'CookTime', 'PrepTime', 'DatePublished', 'Description',
                                    'Images', 'Calories', 'FatContent', 'SaturatedFatContent',
                                    'CholesterolContent', 'SodiumContent', 'CarbohydrateContent',
                                    'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings',
                                    'RecipeYield', 'RecipeInstructions', 'RecipeIngredientParts', 'Keywords',
                                    'RecipeIngredientQuantities', 'ingredient_list'
])
df_sem_atributos

,TotalTime,RecipeCategory,AggregatedRating,ReviewCount
0,PT24H45M,Frozen Desserts,4.5,4.0
1,PT4H25M,Chicken Breast,3.0,1.0
2,PT35M,Beverages,4.5,10.0
3,PT24H20M,Soy/Tofu,4.5,2.0
4,PT50M,Vegetable,4.5,11.0
...,...,...,...,...
522512,PT1H35M,Dessert,NaN,NaN
522513,PT3H30M,Very Low Carbs,NaN,NaN
522514,PT4H,Ice Cream,NaN,NaN
522515,PT15M,Canadian,NaN,NaN


**Tratamento dos campos 'AggregatedRating' e 'ReviewCount':**
* Metade das receitas não possuem avaliações (valores NaN).
* Há 5734 exceções em que há avaliações sem nota atribuída — essas linhas foram removidas.
* Para os demais casos, valores ausentes em 'ReviewCount' e 'AggregatedRating' foram substituídos por 0.
* Essa decisão evita distorções durante o treinamento, já que a média (4.63) e a mediana (5.0) são muito altas.



In [ ]:
df_t = df_sem_atributos.copy()

df_t = df_t[~(df_t['AggregatedRating'].isna() & df_t['ReviewCount'].notna())]

df_t['ReviewCount'] = df_t['ReviewCount'].fillna(0)

df_t['AggregatedRating'] = df_t['AggregatedRating'].fillna(0)

df_t



,TotalTime,RecipeCategory,AggregatedRating,ReviewCount
0,PT24H45M,Frozen Desserts,4.5,4.0
1,PT4H25M,Chicken Breast,3.0,1.0
2,PT35M,Beverages,4.5,10.0
3,PT24H20M,Soy/Tofu,4.5,2.0
4,PT50M,Vegetable,4.5,11.0
...,...,...,...,...
522512,PT1H35M,Dessert,0.0,0.0
522513,PT3H30M,Very Low Carbs,0.0,0.0
522514,PT4H,Ice Cream,0.0,0.0
522515,PT15M,Canadian,0.0,0.0


**Remoção de duplicatas**

In [ ]:
df_t = df_t.drop_duplicates()
df_t

,TotalTime,RecipeCategory,AggregatedRating,ReviewCount
0,PT24H45M,Frozen Desserts,4.5,4.0
1,PT4H25M,Chicken Breast,3.0,1.0
2,PT35M,Beverages,4.5,10.0
3,PT24H20M,Soy/Tofu,4.5,2.0
4,PT50M,Vegetable,4.5,11.0
...,...,...,...,...
522419,PT48H5M,Low Protein,0.0,0.0
522462,PT4H,Thai,0.0,0.0
522489,PT14M,Indian,0.0,0.0
522490,PT23M,Canadian,0.0,0.0


**4. CONVERSÃO DE TEMPO TOTAL PARA MINUTOS**

*   **convert_to_minutes:** Converte uma duração ISO (ex: PT1H30M) em minutos inteiros.

In [ ]:
def convert_to_minutes(s):
  try:
    d = isodate.parse_duration(s)
    return d.total_seconds() / 60
  except:
    return None

df_t['TotalTimeMinutes'] = df_t['TotalTime'].apply(convert_to_minutes)
df_t = df_t[df_t['TotalTimeMinutes'].notna()]
df_t = df_t.drop(columns=['TotalTime'])

len(df_t)
df_t


/tmp/ipython-input-3643240973.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['TotalTimeMinutes'] = df_t['TotalTime'].apply(convert_to_minutes)


,RecipeCategory,AggregatedRating,ReviewCount,TotalTimeMinutes
0,Frozen Desserts,4.5,4.0,1485.0
1,Chicken Breast,3.0,1.0,265.0
2,Beverages,4.5,10.0,35.0
3,Soy/Tofu,4.5,2.0,1460.0
4,Vegetable,4.5,11.0,50.0
...,...,...,...,...
522419,Low Protein,0.0,0.0,2885.0
522462,Thai,0.0,0.0,240.0
522489,Indian,0.0,0.0,14.0
522490,Canadian,0.0,0.0,23.0


**5. GERAÇÃO DO CAMPO “ACCEPTANCESCORE”**
* O campo AcceptanceScore define o gosto do usuário fictício, servindo como a variável alvo para o modelo de regressão.

* A pontuação é calculada definindo um perfil de usuário (uma lista de preferred_categories) e pesos para quatro características principais da receita.

* Funções de pontuação são usadas para normalizar essas características (Tempo, Categoria, Nota e Reviews) em uma escala comum de 0.0 a 1.0.

* O AcceptanceScore final é uma média ponderada, onde as quatro características (Tempo, Categoria, Nota e Reviews) têm, cada uma, 25% de importância na definição do gosto.

* Isso transforma o problema em uma tarefa de regressão em vez de classificação, permitindo ao modelo aprender uma preferência mais detalhada.

In [ ]:
# Categorias preferidas pelo usuário
preferred_categories = ['Desserts', 'Breakfast', '< 30 Mins', 'Vegetables', 'South American', 'Summer', 'Brunch']

def score_total_time_minutes(x):
    return max(0, (60 - x) / 60)

def score_category(x):
  return 1.0 if x in preferred_categories else 0.3

def score_rating(x):
  return min(x/5, 1)

def score_reviews(x):
  return min(x/5, 1)

# Pesos e cálculo da pontuação final
pesos = {
    'TotalTimeMinutes': 0.25,
    'RecipeCategory': 0.25,
    'AggregatedRating': 0.25,
    'ReviewCount': 0.25
}

df_t['AcceptanceScore'] = (
    pesos['TotalTimeMinutes'] * df_t['TotalTimeMinutes'].apply(score_total_time_minutes) +
    pesos['RecipeCategory'] * df_t['RecipeCategory'].apply(score_category) +
    pesos['AggregatedRating'] * df_t['AggregatedRating'].apply(score_rating) +
    pesos['ReviewCount'] * df_t['ReviewCount'].apply(score_reviews)
)

df_t['AcceptanceScore']

,AcceptanceScore
0,0.500000
1,0.275000
2,0.654167
3,0.400000
4,0.591667
...,...
522419,0.075000
522462,0.075000
522489,0.266667
522490,0.229167


**6. TREINAMENTO DO MODELO RANDOM FOREST**

* Divisão da base em treino (80%) e teste (20%):
* O campo 'AcceptanceScore' representa o histórico de aceitação das receitas pelo usuário.
* O modelo supervisionado (Random Forest) aprende padrões de preferência sem depender do estoque atual.
* Essa abordagem permite prever a aceitação de novas receitas e manter o desempenho mesmo com mudanças nos ingredientes disponíveis.
* A avaliação é feita com base na aproximação dos valores previstos e reais de 'AcceptanceScore' (usando RMSE e R²).
* As receitas são recomendadas conforme a ordem decrescente dos valores previstos de aceitação.


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Definição de X (features) e y (target)
x = df_t.drop("AcceptanceScore", axis=1)
y = df_t["AcceptanceScore"]

# Separação por tipo de dado
categorical = x.select_dtypes(include=['object', 'category']).columns.tolist()
numerical = x.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Pré-processamento com codificação de categorias
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical),
        ('numerical', 'passthrough', numerical)
    ]
)

# Criação do pipeline completo com Random Forest
model = Pipeline([
    ('prep', preprocessor),
    ('rf', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Divisão treino/teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, random_state=42)

# Treinamento e avaliação
model.fit(x_treino, y_treino)
y_pred = model.predict(x_teste)

mse = mean_squared_error(y_teste, y_pred)
rmse = np.sqrt(mse)

print("RMSE:", rmse)
print("R²:", r2_score(y_teste, y_pred))
print("duplicated:", df_t.duplicated().sum())


RMSE: 0.007107230390392924
R²: 0.9988255358476217
duplicated: 0


**7. PREPARAÇÃO PRO DF FINAL DA INTERAÇÃO**

Une o df_t com os dados de exibição (Name, ingredient_list) para criar o df final da aplicação.

In [ ]:
df_final_para_interacao = df_t.join(df_sample[['Name', 'ingredient_list']])

# Remove linhas onde a junção falhou ou a lista está vazia
df_final_para_interacao = df_final_para_interacao.dropna(subset=['ingredient_list'])
df_final_para_interacao = df_final_para_interacao[df_final_para_interacao['ingredient_list'].map(lambda l: l != [])]

df_final_para_interacao['ingredient_list'] = df_final_para_interacao['ingredient_list'].apply(tuple)

display(df_final_para_interacao[['Name', 'ingredient_list', 'AcceptanceScore']].head())

,Name,ingredient_list,AcceptanceScore
0,Low-Fat Berry Blue Frozen Dessert,"(blueberries, granulated sugar, vanilla yogurt...",0.500000
1,Biryani,"(saffron, milk, hot green chili peppers, onion...",0.275000
2,Best Lemonade,"(sugar, ""lemons, rind of"", ""lemon, zest of"", f...",0.654167
3,Carina's Tofu-Vegetable Kebabs,"(extra firm tofu, eggplant, zucchini, mushroom...",0.400000
4,Cabbage Soup,"(plain tomato juice, cabbage, onion, carrots, ...",0.591667


**8. INTERAÇÃO COM O USUÁRIO E FILTRAGEM DE RECEITAS**
* **entrada_usuario:** Recebe ingredientes digitados pelo usuário.
* **receitas_possiveis:** Retorna receitas que podem ser preparadas com os ingredientes disponíveis.

In [ ]:
def entrada_usuario():
    print("\nDigite seus ingredientes:")
    print("Quando terminar, digite 'fim'.\n")
    ingredientes_usuario_set = set()
    while True:
        ingr = input("Ingrediente: ").strip().lower()
        if ingr == 'fim':
            break

        ingr = limpar_ingrediente(ingr)

        if ingr:
            ingredientes_usuario_set.add(ingr)
    return ingredientes_usuario_set


def receitas_possiveis(df_receitas, geladeira_usuario_set, modelo, limite_score=0.5):
    receitas_encontradas = []
    features_do_modelo = x.columns.tolist()

    for _, row in df_receitas.iterrows():
        receita_lista = row['ingredient_list']

        if all(ingrediente in geladeira_usuario_set for ingrediente in receita_lista):

            entrada_modelo = pd.DataFrame([row[features_do_modelo]])
            score_previsto = modelo.predict(entrada_modelo)[0]

            if score_previsto >= limite_score:
                receitas_encontradas.append({
                    "Nome da Receita": row.get('Name'),
                    "Aceitação Prevista": round(score_previsto, 3),
                    "Rating": row.get('AggregatedRating'),
                    "Reviews": row.get('ReviewCount'),
                    "Ingredientes da Receita": ", ".join(receita_lista)
                })

    df_resultados = pd.DataFrame(receitas_encontradas)

    if not df_resultados.empty:
        df_resultados = df_resultados.sort_values(by=['Aceitação Prevista', 'Rating'], ascending=False)

    return df_resultados

ingredientes_usuario_set = entrada_usuario()

resultado = receitas_possiveis(df_final_para_interacao, ingredientes_usuario_set, model, limite_score=0.5)

if not resultado.empty:
    print(f"\nReceitas recomendadas ({len(resultado)} encontradas):")
    display(resultado)
else:
    print("\nNenhuma receita atende aos critérios de ingredientes e aceitação mínima (50%).")


Digite seus ingredientes:
Quando terminar, digite 'fim'.

Ingrediente: extra firm tofu
Ingrediente: eggplant
Ingrediente: zucchini
Ingrediente: mushrooms
Ingrediente: soy sauce
Ingrediente: low sodium soy sauce
Ingrediente: olive oil
Ingrediente: maple syrup
Ingrediente: honey
Ingrediente: red wine vinegar
Ingrediente: lemon juice
Ingrediente: garlic cloves
Ingrediente: mustard powder
Ingrediente: black pepper
Ingrediente: fim

Receitas recomendadas (51 encontradas):


,Nome da Receita,Aceitação Prevista,Rating,Reviews,Ingredientes da Receita
32,Greek Yoghurt and Honey,0.996,5.0,7.0,honey
34,Barefoot Contessa's Oven Roasted Bacon,0.908,5.0,126.0,black pepper
4,Homemade Wood Furniture Polish,0.817,5.0,8.0,"olive oil, lemon juice"
15,Harry Potter's Acid Pops,0.804,5.0,7.0,honey
17,Montreal Steak,0.804,5.0,55.0,"olive oil, soy sauce"
43,Asian Sauteed Spinach,0.783,5.0,12.0,"garlic cloves, soy sauce"
5,Barbecued Zucchini-Two Ingredients!,0.771,5.0,10.0,zucchini
14,Grilled Crab Legs,0.758,5.0,20.0,olive oil
7,Marinated Skirt Steak,0.742,5.0,17.0,soy sauce
26,57 Sauce With Honey (For Cocktail Franks),0.738,5.0,9.0,honey
